# Assignment: Ionosphere Data Problem

### Dataset Description: 

This radar data was collected by a system in Goose Bay, Labrador. This system consists of a phased array of 16 high-frequency antennas with a total transmitted power on the order of 6.4 kilowatts. See the paper for more details. The targets were free electrons in the ionosphere. "Good" radar returns are those showing evidence of some type of structure in the ionosphere. "Bad" returns are those that do not; their signals pass through the ionosphere.

Received signals were processed using an autocorrelation function whose arguments are the time of a pulse and the pulse number. There were 17 pulse numbers for the Goose Bay system. Instances in this databse are described by 2 attributes per pulse number, corresponding to the complex values returned by the function resulting from the complex electromagnetic signal.

### Attribute Information:

- All 34 are continuous
- The 35th attribute is either "good" or "bad" according to the definition summarized above. This is a binary classification task.

 <br><br>

<table border="1"  cellpadding="6">
	<tbody>
        <tr>
		<td bgcolor="#DDEEFF"><p class="normal"><b>Data Set Characteristics:&nbsp;&nbsp;</b></p></td>
		<td><p class="normal">Multivariate</p></td>
		<td bgcolor="#DDEEFF"><p class="normal"><b>Number of Instances:</b></p></td>
		<td><p class="normal">351</p></td>
		<td bgcolor="#DDEEFF"><p class="normal"><b>Area:</b></p></td>
		<td><p class="normal">Physical</p></td>
        </tr>
     </tbody>
    </table>
<table border="1" cellpadding="6">
    <tbody>
        <tr>
            <td bgcolor="#DDEEFF"><p class="normal"><b>Attribute Characteristics:</b></p></td>
            <td><p class="normal">Integer,Real</p></td>
            <td bgcolor="#DDEEFF"><p class="normal"><b>Number of Attributes:</b></p></td>
            <td><p class="normal">34</p></td>
            <td bgcolor="#DDEEFF"><p class="normal"><b>Date Donated</b></p></td>
            <td><p class="normal">N/A</p></td>
        </tr>
     </tbody>
    </table>
<table border="1" cellpadding="6">	
    <tbody>
    <tr>
		<td bgcolor="#DDEEFF"><p class="normal"><b>Associated Tasks:</b></p></td>
		<td><p class="normal">Classification</p></td>
		<td bgcolor="#DDEEFF"><p class="normal"><b>Missing Values?</b></p></td>
		<td><p class="normal">N/A</p></td>
		<td bgcolor="#DDEEFF"><p class="normal"><b>Number of Web Hits:</b></p></td>
		<td><p class="normal">N/A</p></td>
	</tr>
    </tbody>
    </table>

### WORKFLOW :
- Load Data
- Check Missing Values ( If Exist ; Fill each record with mean of its feature ) or any usless column.
- Shuffle the data if needed.
- Standardized the Input Variables. **Hint**: Centeralized the data
- Split into 60 and 40 ratio.
- Encode labels.
- Model : 1 hidden layers including 16 unit.
- Compilation Step (Note : Its a Binary problem , select loss , metrics according to it)
- Train the Model with Epochs (100).
- If the model gets overfit tune your model by changing the units , No. of layers , epochs , add dropout layer or add Regularizer according to the need .
- Prediction should be > **92%**
- Evaluation Step
- Prediction


## **Loading Data**

In [2]:
from google.colab import files 

uploaded = files.upload()

Saving ionosphere_data.csv to ionosphere_data.csv


In [3]:
import pandas as pd
import numpy as np

dataframe = pd.read_csv('ionosphere_data.csv')

## **Checking for Missing Values**

In [4]:
dataframe.isnull().any().any()

False

## **Shuffling our Dataset**

In [5]:
dataframe_shuffled = pd.DataFrame(np.random.permutation(dataframe) , columns = dataframe.columns)

In [6]:
dataframe_shuffled.head()

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,feature11,feature12,feature13,feature14,feature15,feature16,feature17,feature18,feature19,feature20,feature21,feature22,feature23,feature24,feature25,feature26,feature27,feature28,feature29,feature30,feature31,feature32,feature33,feature34,label
0,1,0,0.94531,-0.03516,-1,-0.33203,-1,-0.01563,0.97266,0.01172,0.93359,-0.01953,-1,0.16406,-1,-0.00391,0.95313,-0.03516,0.92188,-0.02734,-0.99219,0.11719,-0.93359,0.34766,0.95703,-0.00391,0.82041,0.13758,0.90234,-0.06641,-1,-0.1875,-1,-0.34375,b
1,1,0,0.62745,-0.07843,0.72549,0,0.60784,-0.07843,0.62745,-0.11765,0.68627,-0.11765,0.66667,-0.13725,0.64706,-0.09804,0.54902,-0.11765,0.54902,-0.21569,0.58824,-0.19608,0.66667,-0.23529,0.45098,-0.2549,0.52409,-0.24668,0.56863,-0.31373,0.43137,-0.21569,0.47059,-0.27451,b
2,0,0,1,1,0,0,1,-1,1,-1,1,1,1,1,1,-1,1,1,1,1,1,-1,-1,-1,1,-1,1,-1,1,1,0,0,1,-1,b
3,1,0,0,0,1,0.51724,0,0,0.10991,-1,0,0,0,0,-1,-0.22414,-0.55711,-0.83297,0.7694,0.63147,0,0,0.53448,0.35668,-0.90302,0.44828,1,-1,-1,0.81573,0,0,0,0,b
4,1,0,0.41932,0.12482,0.35,0.125,0.23182,0.27955,-0.03636,0.44318,0.04517,0.36194,-0.19091,0.33636,-0.1335,0.27322,0.02727,0.40455,-0.34773,0.12727,-0.20028,0.05078,-0.18636,0.36364,-0.14003,-0.04802,-0.09971,-0.07114,-1,-1,-0.02916,-0.07464,-0.00526,-0.06314,b


## **Preprocessing Data**

### Removing unnecessary features

In [7]:
data = dataframe_shuffled.drop(['label', 'feature2'], axis = 1)
data.shape

(351, 33)

### Converting labels to numbers

In [8]:
target_labels = dataframe_shuffled['label'].map({'b' : 0 , 'g' : 1})
target_labels[:20]

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     1
10    1
11    1
12    1
13    0
14    0
15    1
16    1
17    1
18    1
19    1
Name: label, dtype: int64

In [9]:
target_labels = np.array(target_labels)

In [10]:
target_labels.shape

(351,)

## **Splitting Data**

In [11]:
train_len = len(dataframe)*6//10
test_len = len(dataframe) - train_len

In [12]:
train_labels = target_labels[0 : train_len].copy()
test_labels = target_labels[train_len : train_len + test_len].copy()

In [13]:
train_labels.shape, test_labels.shape

((210,), (141,))

In [14]:
train_labels.dtype

dtype('int64')

In [15]:
train_labels = train_labels.astype('float32')

In [16]:
train_data = data[0 : train_len].copy()
test_data = data[train_len : train_len + test_len].copy()

In [17]:
train_data.shape, test_data.shape

((210, 33), (141, 33))

In [18]:
train_data = np.array(train_data).astype('float32')
test_data = np.array(test_data).astype('float32')

## **Normalizing Data**

In [19]:
mean = np.mean(train_data, axis=0)
std = np.std(train_data,axis=0)


In [20]:
train_data -= mean
train_data /= std

In [21]:
test_data -= mean
test_data /= std

In [22]:
train_data.shape, test_data.shape , train_labels.shape, test_labels.shape

((210, 33), (141, 33), (210,), (141,))

## **Building & Compiling Model in a function**

In [27]:
# Building a model + K-Fold Validation:

from keras import models
from keras import layers
from keras import regularizers
def build_model():
  modelI = models.Sequential()
  modelI.add(layers.Dense(64, activation='relu', kernel_regularizer = regularizers.l2(0.001) ,input_shape = (train_data.shape[1],) ) )  
  modelI.add(layers.Dropout(0.3))
  modelI.add(layers.Dense(16,  activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
  modelI.add(layers.Dropout(0.3))
  modelI.add(layers.Dense(1 , activation = 'sigmoid'))

  modelI.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
  return modelI

## **Validating our Approach & Training Model**

In [28]:
k = 5
number_of_validation_samples = len(train_data)//k

number_of_epochs = 100
all_scores = []

for i in range(k):
  print('processing fold #', i)
  val_data = train_data[i*number_of_validation_samples : (i+1) * number_of_validation_samples]        # Prepares the validation data from partition #k
  val_labels = train_labels[i*number_of_validation_samples : (i+1) * number_of_validation_samples]

  partial_train_data = np.concatenate( ( train_data[ : i*number_of_validation_samples],              # Prepares the training data from all other partitions
                                         train_data[(i+1)*number_of_validation_samples : ] ) , axis = 0 )
  
  partial_train_labels = np.concatenate( ( train_labels[ : i*number_of_validation_samples], 
                                         train_labels[(i+1)*number_of_validation_samples : ] ) , axis = 0 )
  
  modelI = build_model()  # builds keras model (already compiled) --> remember the function we made

  historyI = modelI.fit(partial_train_data,
             partial_train_labels,
             epochs = number_of_epochs,
             batch_size =  16, 
             verbose = 1,  # -->    verbose: 0, 1, or 2  --> 0 = silent, 1 = progress bar, 2 = one line per epoch. 
             validation_data = (val_data, val_labels) )


processing fold # 0
Epoch 1/100
11/11 [==============================] - 1s 21ms/step - loss: 0.7487 - accuracy: 0.5262 - val_loss: 0.6254 - val_accuracy: 0.6667
Epoch 2/100
11/11 [==============================] - 0s 5ms/step - loss: 0.6725 - accuracy: 0.6649 - val_loss: 0.5949 - val_accuracy: 0.7857
Epoch 3/100
11/11 [==============================] - 0s 5ms/step - loss: 0.5592 - accuracy: 0.8107 - val_loss: 0.5733 - val_accuracy: 0.7857
Epoch 4/100
11/11 [==============================] - 0s 5ms/step - loss: 0.5600 - accuracy: 0.8021 - val_loss: 0.5588 - val_accuracy: 0.7857
Epoch 5/100
11/11 [==============================] - 0s 6ms/step - loss: 0.5308 - accuracy: 0.7892 - val_loss: 0.5484 - val_accuracy: 0.8095
Epoch 6/100
11/11 [==============================] - 0s 6ms/step - loss: 0.4093 - accuracy: 0.8761 - val_loss: 0.5308 - val_accuracy: 0.8333
Epoch 7/100
11/11 [==============================] - 0s 6ms/step - loss: 0.3936 - accuracy: 0.8987 - val_loss: 0.5139 - val_accuracy:

## **Finalizing Model & Evaluating on Test Data**

In [29]:
model = build_model()
model.fit(train_data, train_labels,
          epochs = 40, batch_size = 16, verbose = 0)

test_loss , test_accuracy = model.evaluate(test_data, test_labels)


5/5 [==============================] - 0s 2ms/step - loss: 0.2356 - accuracy: 0.9220


In [30]:
test_accuracy

0.9219858050346375